In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Conv2DTranspose, Reshape, BatchNormalization, Flatten, Dropout, Input, MaxPooling2D
from keras.initializers import RandomNormal
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.applications.resnet import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import matplotlib.pyplot as plt
import os
import numpy as np
import time
from google.colab import drive
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,recall_score
import itertools
import warnings
from sklearn import metrics
warnings.filterwarnings('ignore')


In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Load train dataset

In [ ]:
train_folder = ""
train_data = image_dataset_from_directory(train_folder,
    label_mode="binary",
    labels="inferred",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True)

rescale = Rescaling(scale=1.0/255)
train_data = train_data.map(lambda image,label:(rescale(image),label))

Load test dataset

In [ ]:
test_folder =  ""
test_data = image_dataset_from_directory(test_folder,
    label_mode="binary",
    labels="inferred",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True)

rescale = Rescaling(scale=1.0/255)
test_data = test_data.map(lambda image,label:(rescale(image),label))

Import pretrained model

In [ ]:
pretrained_model = VGG16(include_top=False, input_shape=(128,128,3))
for layer in pretrained_model.layers:
  layer.trainable=False

Build CNN

In [ ]:
model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
model.summary()

Define training callbacks

In [ ]:
checkpoint_path = ""
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10)

Train model

In [ ]:
history = model.fit(  
    train_data,
    epochs=100,
    validation_data=test_data,
    callbacks=[model_checkpoint_callback, early_stopping]
)

Plot training graphs

In [ ]:
acc = list(map(lambda x: x * 100, history.history['accuracy']))
val = list(map(lambda x: x * 100, history.history['val_accuracy']))
plt.plot(acc)
plt.plot(val)
plt.title('Model Accuracy')
plt.ylabel('accuracy (%)')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Compute metrics

In [ ]:
model.load_weights(checkpoint_path)
pred = []
lab =  []
for x, y in test_data:
  pred.append(np.round(model.predict(x)))
  lab.append(y.numpy())
  
predictions = np.concatenate(pred)
labels = np.concatenate(lab)

In [ ]:
print(metrics.accuracy_score(labels, predictions))

In [ ]:
print(classification_report(predictions,  labels, digits=4))

Plot confusion matrix

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
                          
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')   

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(predictions,  labels)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(5, 5))
plot_confusion_matrix(cnf_matrix, classes=['covid', 'normal'],
                      title='Confusion matrix')
plt.show()



Compute evaluation metrics

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [ ]:
TP, FP, TN, FN = perf_measure(predictions, labels)

Plot ROC curve

In [ ]:
fpr, tpr, thresholds= metrics.roc_curve(predictions, labels)
auc = metrics.auc(fpr, tpr)

In [ ]:
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC (area = {:.3f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()